## Libraries

In [218]:
!pip install numpy
!pip install pandas
!pip install numpy
!pip install matplotlib

import matplotlib.pyplot as plt

import sklearn.metrics as sklm
import copy
import json

import pandas as pd
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utils

In [219]:
PATH = "./"
ROOT_PATH = "../../"

In [220]:
def retrieve_data(flow_interval):
    path = "{0}dataset/dataset_flow_{1}.csv".format(ROOT_PATH, flow_interval)
    print(PATH)
    data = pd.read_csv(path, ';')
    
    data['Flow'].apply(int)
    data['AveSpeed'].apply(float)
    data['Density'].apply(float)
    data['Sunday'].apply(int)
    data['Monday'].apply(int)
    data['Tuesday'].apply(int)
    data['Wednesday'].apply(int)
    data['Thursday'].apply(int)
    data['Friday'].apply(int)
    data['Saturday'].apply(int)
      
    return data

In [221]:
def print_json (obj):
  print(json.dumps(obj, sort_keys=True, indent=4))

In [222]:
def store(obj, path, name):
  with open("{0}{1}/{2}.json".format(PATH, path, name), 'w') as json_file:
    json.dump(obj, json_file, sort_keys=True, indent=4)

## Testing

In [223]:
def plot_prediction (Y, Y_hat, title, alpha, gamma):
  """ Plot Prediction
  
  Plot the prediction (Flow x Time) of what was expected and what
  was predicted.
  """
  alpha_dec = str(alpha - int(alpha)).split('.')[1]
  gamma_dec = str(gamma - int(gamma)).split('.')[1]
  name = f"{title}_alpha_d{alpha_dec}_gamma_d{gamma_dec})"
  path = f"plots/prediction/{name}"
    
  plt.plot(Y)
  plt.plot(Y_hat)
  plt.title(f"Predição do Modelo {title} (alpha {alpha}, gamma {gamma})")
  plt.ylabel('Fluxo')
  plt.xlabel('Tempo')
  plt.legend(['esperado', 'observado'], loc='upper left')
  plt.rcdefaults()

  plt.savefig(path + ".png", bbox_inches='tight')
  # plt.savefig(path + ".pdf")

  plt.close('all')

In [224]:
# Model Parameters
FLOW_INTERVAL = 150 # the interval size for each flow
ALPHAS = [0.001, 0.01, 0.1, 0.2, 0.5, 0.8]
GAMMAS = [0.001, 0.01, 0.1, 0.2, 0.5, 0.8]

In [225]:
def double_exponential_smoothing(X, alpha, gamma):
  global result_data
  
  name = "DES"
  expected, observed = [], []
  W = 1

  X_test = np.array_split(X, 10)
  F = X_test[8][-1]
  Lm1 = F
  bm1 = 0
  for x in X_test[9]:
    L = (alpha * x) + ( (1 - alpha) * (Lm1 + bm1) )
    b = ( gamma * (L - Lm1) ) + ( (1 - gamma) * bm1 )
    F1 = L + (b*W)
    
    expected.append(int(x))
    observed.append(float(F))
    F = F1
    Lm1 = L
    bm1 = b
  
  result_data['results']['RMSE'] = sklm.mean_squared_error(expected, observed, squared=False)
  result_data['results']['expected'] = expected
  result_data['results']['observed'] = observed
  alpha_dec = str(alpha - int(alpha)).split('.')[1]
  gamma_dec = str(gamma - int(gamma)).split('.')[1]
  store(result_data, "results", "{0}_alpha_d{1}_gamma_d{2}".format(name, alpha_dec, gamma_dec))
  # store(result_data, "results", f"{name}_alpha_d{alpha_dec}_gamma_d{gamma_dec}")

In [226]:
global result_data

data = retrieve_data(FLOW_INTERVAL)
sequence = np.array(data['Flow'])

for alpha, gamma in [(x,y) for x in ALPHAS for y in GAMMAS]:
  print("ALPHA: {0}".format(alpha))
  print("GAMMA: {0}".format(gamma))
  result_data = {
    'results': {},
    'meta': {
      'FLOW_INTERVAL': FLOW_INTERVAL,
    }
  }
  
  double_exponential_smoothing(sequence, alpha, gamma)
  print("RSME: {0}".format(result_data['results']['RMSE']))
  
  expected = result_data['results']['expected']
  observed = result_data['results']['observed']
  plot_prediction(expected, observed, "DES", alpha, gamma)
  print("####################################################")

./
ALPHA: 0.001
GAMMA: 0.001
RSME: 9.089171078193255
####################################################
ALPHA: 0.001
GAMMA: 0.01
RSME: 9.201481282623503
####################################################
ALPHA: 0.001
GAMMA: 0.1
RSME: 31.942577088252037
####################################################
ALPHA: 0.001
GAMMA: 0.2
RSME: 13.910295736312252
####################################################
ALPHA: 0.001
GAMMA: 0.5
RSME: 26.108981024358357
####################################################
ALPHA: 0.001
GAMMA: 0.8
RSME: 10.262390869515714
####################################################
ALPHA: 0.01
GAMMA: 0.001
RSME: 7.9062471813051145
####################################################
ALPHA: 0.01
GAMMA: 0.01
RSME: 9.034479343984517
####################################################
ALPHA: 0.01
GAMMA: 0.1
RSME: 7.2856825600653
####################################################
ALPHA: 0.01
GAMMA: 0.2
RSME: 6.864752332538132
###################################